## Prototype Pyresample AreaDefinition to_cf()

This notebook is a prototype for a routine that is the counterpart of [load_cf_area()](https://pyresample.readthedocs.io/en/latest/howtos/geometry_utils.html#loading-from-netcdf-cf). It takes a pyresample AreaDefinition object as input and returns an xarray Dataset that holds the basic information needed to prepare netCDF/CF files for this particular grid. It returns a template that can be further worked upon (add relevant variables, more attributes, more dimensions, etc...).

Status:
* Sept 7th 2023 : Prototype ready for inspection. It does not support all possible CF projections. It also does not support user configuration (e.g. via a cf_info).
* Sept 8th 2023 : Move encodings to the DataArrays (doesn't seem to work for coordinate variables lat/lon)
* Sept 28th 2023 : Add a small chunk of code to decide the order of x and y dimensions from the shape of lat/lon

TODO :
* Investigate if the newer pyproj's [crs.cs_to_cf()](https://pyproj4.github.io/pyproj/latest/build_crs_cf.html) could be used to shorten this even more.

In [1]:
import numpy as np
import xarray as xr
import pyresample as pr

In [2]:
# load an area_def from a netCDF/CF file (could be any other solution to obtain an AreaDefinition)
ds = xr.open_dataset('https://thredds.met.no/thredds/dodsC/osisaf/met.no/reprocessed/ice/conc_450a_files/monthly/2020/ice_conc_nh_ease2-250_cdr-v3p0_202011.nc')
area_def, cf_info = pr.utils.load_cf_area(ds)

/home/thomasl/mambaforge/envs/py310/lib/python3.10/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/thomasl/mambaforge/envs/py310/lib/python3.10/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [3]:
def to_cf_template(area_def, skip_lonlat=True, crs_encoding='pyproj_to_cf'):
    """Return a template xarray Dataset holding the structure of a netCDF/CF file for this grid."""
    
    # prepare the crs object with pyproj.to_cf()
    crs_cf = area_def.crs.to_cf()
    type_of_grid_mapping = crs_cf['grid_mapping_name']
    
    # alter this set of attributes on user input
    if crs_encoding == 'pyproj_to_cf':
        pass
    elif crs_encoding == 'only_cf_attrs':
        crs_cf.pop('crs_wkt')
    elif crs_encoding == 'only_wkt':
        crs_cf = {'crs_wkt': crs_cf['crs_wkt']}
    else:
        raise ValueError("Unsupported value for crs_encoding=")
        
    # prepare the x and y axis (1D)
    xy = dict()
    xy_dims = ('x', 'y')
    for axis in xy_dims:
        
        # access the valid standard_names (also handle the 'default')
        try:
            valid_coord_standard_names = pr.utils.cf._valid_cf_coordinate_standardnames[type_of_grid_mapping][axis]
        except KeyError:
            valid_coord_standard_names = pr.utils.cf._valid_cf_coordinate_standardnames['default'][axis]
        
        xy[axis] = dict()
        # CF wants the center of the grid cells
        if axis == 'x':
            xy[axis]['_coords'] = area_def.projection_x_coords
        else:
            xy[axis]['_coords'] = area_def.projection_y_coords     
        # each axis requires a valid name, which depends on the type of projection
        xy[axis]['standard_name'] = valid_coord_standard_names[0]
        # CF recommendation to have axis= attribute
        xy[axis]['axis'] = axis.upper()
        # units
        xy[axis]['units'] = 'm'
    
    # latitude and longitude (2D)
    lons, lats = area_def.get_lonlats()
    
    # determine the order of the x, y dimensions that match the shape of lat/lon arrays
    
    if lons.shape == (len(xy['y']['_coords']),len(xy['x']['_coords'])):
        xy_dims = ('y', 'x')
    elif lons.shape == (len(xy['x']['_coords']),len(xy['y']['_coords'])):
        xy_dims = ('x', 'y')
    else:
        raise ValueError("Incompatible shape for lon/lat {}, x {}, and y {}.".format(lons.shape,
                                                                                    len(xy['x']['_coords']),
                                                                                    len(xy['y']['_coords'])))
    
    # define a Dataset as a template.
    #   we cannot define a Dataset without a data variable. The strategy is to 
    #   create the Dataset with an (empty) variable, and the user can later 
    #   add his own variables, and finally drop our 'template' variable before
    #   writing to file.
    varn = 'template'
    shape = lons.shape
    da_empty_data = np.ones_like(lons) * np.nan
    da_dims = list(xy_dims)
    da_coords = {'x':('x',xy['x']['_coords']), 'y':('y',xy['y']['_coords']),}
    if not skip_lonlat:
        da_coords['lon']=(xy_dims, lons)
        da_coords['lat']=(xy_dims, lats)
    ds = xr.Dataset(data_vars={varn:(da_dims, da_empty_data),
                               'crs':([], 0)},
                    coords=da_coords,)
    
    # add CF attributes and encodings to the xarray template
    
    # x and y dims
    for axis in xy_dims:
        for attr in xy[axis].keys():
            if attr.startswith('_'): continue
            ds[axis].attrs[attr] = xy[axis][attr]
        ds[axis].encoding = {'_FillValue':None}
    
    # crs object
    ds['crs'].attrs = crs_cf
    ds['crs'].encoding = {'dtype':'int32'}
    
    # latitude and longitude
    if not skip_lonlat:
        ds['lon'].attrs={'long_name': 'longitude coordinate', 'standard_name': 'longitude', 'units': 'degrees_east'}
        ds['lat'].attrs={'long_name': 'latitude coordinate', 'standard_name': 'latitude', 'units': 'degrees_north'}
        ds['lon'].encoding = {'_FillValue':None}
        ds['lat'].encoding = {'_FillValue':None}
        
    # the empty variable itself
    ds[varn].attrs['grid_mapping'] = 'crs'
    
    # global attributes
    ds.attrs['Conventions'] = 'CF-1.8'
    
    return ds
    

In [4]:
# Demonstrate the routine.

# Use to_cf_template() to obtain a template for the netCDF/CF file
ds = to_cf_template(area_def, skip_lonlat=False, )

# Create a DataArray holding user data that we want to add to the file. 
myData = np.arange(0,ds['lon'].size).reshape(ds['lon'].shape).astype('float')
da_var2 = xr.DataArray(myData, coords=ds['template'].coords, dims=ds['template'].dims,
                       attrs=ds['template'].attrs, name='my_var')
# Add some attributes
da_var2.attrs['long_name'] = 'my variable defined on that particular grid'
da_var2.attrs['units'] = 'K'

# Add the variable to the template file
ds = ds.merge(da_var2)

# We don't need the template variable anymore, drop it.
ds = ds.drop('template')

# Write to file.
o_nc = 'cf_template.nc'
ds.to_netcdf('cf_template.nc', format='NETCDF4_CLASSIC')

In [5]:
!ncdump -ch cf_template.nc

netcdf cf_template {
dimensions:
	x = 432 ;
	y = 432 ;
variables:
	int crs ;
		crs:crs_wkt = "PROJCRS[\"undefined\",BASEGEOGCRS[\"undefined\",DATUM[\"undefined\",ELLIPSOID[\"undefined\",6378137,298.257223563,LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]]],PRIMEM[\"Greenwich\",0,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8901]]],CONVERSION[\"unknown\",METHOD[\"Lambert Azimuthal Equal Area\",ID[\"EPSG\",9820]],PARAMETER[\"Latitude of natural origin\",90,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8801]],PARAMETER[\"Longitude of natural origin\",0,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8802]],PARAMETER[\"False easting\",0,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8806]],PARAMETER[\"False northing\",0,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8807]]],CS[Cartesian,2],AXIS[\"(E)\",east,ORDER[1],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]],AXIS[\"(N)\",north,ORDER[2],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]]]" ;
		crs:semi_major_axis = 6378137. ;
		crs:semi_minor_axis = 635675

In [6]:
# Simple test: can we load the AreaDefinition from the file?
new_area_def, new_cf_info = pr.utils.load_cf_area(o_nc)
# If yes, is it the same AreaDefinition as we wrote in?
if (area_def != new_area_def):
    print("EQUALITY AREA_EXTENT: ", np.allclose(area_def.area_extent, new_area_def.area_extent))
    print("EQUALITY SHAPE: ", area_def.shape == new_area_def.shape)
    print("EQUALITY CRS: ", area_def.crs == new_area_def.crs)
    lons, lats = area_def.get_lonlats()
    nlons, nlats = new_area_def.get_lonlats()
    print("EQUALITY LATs: ", abs(lats-nlats).max() < 1.e-4)
    print("EQUALITY LONs: ", abs(lons-nlons).max() < 1.e-4)
else:
    print("EQUALITY")

EQUALITY


/home/thomasl/mambaforge/envs/py310/lib/python3.10/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
